In [1]:
import numpy as np
import os
from pynwb import NWBFile, NWBHDF5IO, image
from pynwb.ophys import TwoPhotonSeries, OpticalChannel, ImageSegmentation, ImagingPlane
from pynwb.file import Subject
from pynwb.device import Device
import math

import statistics
import pandas as pd
import h5py #Provides methods to supplement NWB
import datajoint as dj #
import csv 
from datetime import datetime
from dateutil import tz
from pynwb import NWBHDF5IO
from matplotlib import pylab as plt

def connect_to_dj():
    dj.config['database.host'] = 'spinup-db001f1f.cluster-cyynsscieqtk.us-east-1.rds.amazonaws.com' 
    dj.config['database.user'] = 'MarikeReimer'
    dj.config['database.password'] = 'uqHKL3YLMCG0'
    dj.conn()

    #Create a schema to organize the pipeline. Defining it here means you only need to change the code in one place.
    current_schema = dj.schema('MarikeReimer', locals())
    return current_schema


schema = connect_to_dj()
print(schema)
schema

#Create a schema to organize the pipeline. Defining it here means you only need to change the code in one place.
schema = dj.schema('MarikeReimer', locals())
schema


#Define Mouse table
@schema
class Mouse(dj.Manual):
    definition = """
    subject_id: varchar(128)                  # Primary keys above the '---'
    ---
    #non-primary columns below the '---' 
    genotype: enum('B6', 'BalbC', 'Unknown', 'Thy1-YFP')
    sex: enum('M', 'F', 'Unknown')
    species: varchar(128)
    strain: varchar(128)
    """

@schema
class Session(dj.Manual):
    definition = """
    ->Mouse
    identifier: varchar(128)                  # Primary keys above the '---'
    ---
    #non-primary columns below the '---' 
    surgery: varchar(128)
    pharmacology: varchar(128)
    """


@schema
class Dendrite(dj.Manual):
    definition = """
    ->Session
    dendrite_id: int                  # Primary keys above the '---'
    ---
    #non-primary columns below the '---'
    soma_center_point: longblob
    proximal_dendrite_length: float
    medial_dendrite_length: float
    distal_dendrite_length: float
    """

@schema
class Image_segmentation(dj.Imported):
    definition = """
    ->Dendrite
    segmentation_name: varchar(128)                  # Primary keys above the '---'
    ---
    #non-primary columns below the '---'
    length: float
    volume: float
    surface_area:float
    spine_type: enum('mushroom', 'thin', 'disconnected', 'stubby', 'U')
    center_of_mass: longblob
    """
    def make(self, key):
        subject_id = key['subject_id']
        identifier = key['identifier']

        nwbfile_to_read = 'C:/Users/meowm/OneDrive/TanLab/DataJointTesting/NWBfiles/' + str(subject_id) + str(identifier) + '.nwb' #TODO: Remove hard coding
        #nwbfile_to_read = 'C:/Users/meowm/Downloads/NWBfiles/' + str(subject_id) + str(identifier) + '.nwb'

        print(nwbfile_to_read)
        with NWBHDF5IO(nwbfile_to_read, 'r') as io:
            nwbfile = io.read()     
            for group in nwbfile.processing["SpineData"]["ImageSegmentation"].children[:]:
                #print(group.name)
                if group.name.startswith("Mushroom"):
                    spine_type = 'mushroom'
                elif group.name.startswith("Thin"):
                    spine_type = 'thin'
                elif group.name.startswith("Disconnected"):
                    spine_type = 'disconnected'
                elif group.name.startswith("Stubby"):
                    spine_type = 'stubby'
                else:
                    spine_type = 'U'

                length = nwbfile.processing["SpineData"]["ImageSegmentation"][group.name].length.data[:]
                length = length[0]
                volume = nwbfile.processing["SpineData"]["ImageSegmentation"][group.name].volume.data[:]
                volume = volume[0]
                surface_area = nwbfile.processing["SpineData"]["ImageSegmentation"][group.name].surface_area.data[:]
                surface_area = surface_area[0]
                center_of_mass = nwbfile.processing["SpineData"]["ImageSegmentation"][group.name].center_of_mass.data[:]
                center_of_mass = center_of_mass[0]
                
                key['segmentation_name'] = group.name 
                key['length'] = length
                key['volume'] = volume
                key['surface_area'] = surface_area
                key['spine_type'] = spine_type
                key['center_of_mass'] = center_of_mass
                self.insert1(key)

@schema
class Distance_to_soma(dj.Computed):
    definition = """
    ->Image_segmentation
    ---
    distance_to_soma: float"""
    def make(self, key):
        center_of_mass = (Image_segmentation() & key).fetch1('center_of_mass')
        soma_center_point = (Dendrite() & key).fetch1('soma_center_point')
        distance_to_soma = math.dist(center_of_mass,soma_center_point)

        key['distance_to_soma'] = distance_to_soma
        self.insert1(key)

#Instantiate tables
mouse = Mouse()
session = Session()
dendrite = Dendrite()
image_segmentation = Image_segmentation()
distance_to_soma = Distance_to_soma()

print(distance_to_soma)

[2023-08-22 10:47:26,053][INFO]: Connecting MarikeReimer@spinup-db001f1f.cluster-cyynsscieqtk.us-east-1.rds.amazonaws.com:3306
[2023-08-22 10:47:26,557][INFO]: Connected MarikeReimer@spinup-db001f1f.cluster-cyynsscieqtk.us-east-1.rds.amazonaws.com:3306


Schema `MarikeReimer`

*subject_id    *identifier    *dendrite_id   *segmentation_ distance_to_so
+------------+ +------------+ +------------+ +------------+ +------------+
L684           Dendrite1a     11             DisconnectedMu 107.769       
L684           Dendrite1a     11             StubbySpine_0_ 17.7509       
L684           Dendrite1a     11             StubbySpine_10 104.197       
L684           Dendrite1a     11             StubbySpine_11 75.9831       
L684           Dendrite1a     11             StubbySpine_12 75.3099       
L684           Dendrite1a     11             StubbySpine_15 89.2687       
L684           Dendrite1a     11             StubbySpine_18 107.521       
L684           Dendrite1a     11             StubbySpine_1_ 29.4164       
L684           Dendrite1a     11             StubbySpine_2_ 54.7994       
L684           Dendrite1a     11             StubbySpine_3_ 63.3258       
L684           Dendrite1a     11             StubbySpine_4_ 68.2698       
L6

In [44]:
#drop all the tables
mouse.drop()

[2023-08-15 09:41:54,137][INFO]: `MarikeReimer`.`mouse` (18 tuples)
[2023-08-15 09:41:54,291][INFO]: `MarikeReimer`.`session` (65 tuples)
[2023-08-15 09:41:54,436][INFO]: `MarikeReimer`.`dendrite` (65 tuples)
[2023-08-15 09:41:54,602][INFO]: `MarikeReimer`.`_image_segmentation` (1776 tuples)
[2023-08-15 09:41:54,757][INFO]: `MarikeReimer`.`__distance_to_soma` (1776 tuples)
[2023-08-15 09:41:56,978][INFO]: Dropped table `MarikeReimer`.`__distance_to_soma`
[2023-08-15 09:41:57,309][INFO]: Dropped table `MarikeReimer`.`_image_segmentation`
[2023-08-15 09:41:57,625][INFO]: Dropped table `MarikeReimer`.`dendrite`
[2023-08-15 09:41:57,965][INFO]: Dropped table `MarikeReimer`.`session`
[2023-08-15 09:41:58,300][INFO]: Dropped table `MarikeReimer`.`mouse`
[2023-08-15 09:41:58,539][INFO]: Tables dropped. Restart kernel.


In [ ]:
#For general load checking
mouse*session

In [56]:
#Check if a specific subject loaded
session & 'subject_id = "L712"'

subject_id Primary keys above the '---',identifier Primary keys above the '---',surgery,pharmacology
L712,Dendrite1a,Contusion,Romidepsin 2.5mg/kg inject IP 4 weeks after SCI
L712,Dendrite1b,Contusion,Romidepsin 2.5mg/kg inject IP 4 weeks after SCI
L712,Dendrite2,Contusion,Romidepsin 2.5mg/kg inject IP 4 weeks after SCI
L712,Dendrite3,Contusion,Romidepsin 2.5mg/kg inject IP 4 weeks after SCI


In [5]:
#Delete a Subject with an identifier - use after failing QA on a Blender file
(session & 'subject_id = "L882"' & 'identifier = "Dendrite5F2"').delete()

[2023-08-22 11:12:51,884][INFO]: Deleting 25 rows from `MarikeReimer`.`__distance_to_soma`
[2023-08-22 11:12:52,173][INFO]: Deleting 25 rows from `MarikeReimer`.`_image_segmentation`
[2023-08-22 11:12:52,434][INFO]: Deleting 1 rows from `MarikeReimer`.`dendrite`
[2023-08-22 11:12:52,712][INFO]: Deleting 1 rows from `MarikeReimer`.`session`
[2023-08-22 11:12:55,047][INFO]: Deletes committed.


1

In [6]:
#Display info for a Subject with an identifier 

(Session & image_segmentation & distance_to_soma & 'subject_id = "L736"' & 'identifier = "Dendrite1"')

subject_id Primary keys above the '---',identifier Primary keys above the '---',surgery,pharmacology
L736,Dendrite1,Contusion,Vehicle 1% DMSO inject IP 4 weeks after SCI


In [69]:
#Pivot tables in Excel exclude blank values, so I download this separately to make sure spineless dendrites don't get missed
frame = pd.DataFrame(mouse*session*dendrite)
pd.DataFrame(frame).to_csv("C:/Users/meowm/Downloads/SubjList.csv")

In [70]:
#Export pipeline to csv
frame = pd.DataFrame(mouse*session*dendrite*image_segmentation*distance_to_soma)
pd.DataFrame(frame).to_csv("C:/Users/meowm/Downloads/UnfilteredDiscData_V1_dendritestart.csv")

In [ ]:
#View pipeline
mouse*session*dendrite*image_segmentation*distance_to_soma

In [7]:
#Query for dataset 1.3
frame = pd.DataFrame(mouse*(session & 'pharmacology = "Vehicle 1% DMSO inject IP 4 weeks after SCI"')*dendrite*(image_segmentation & 'volume >= "0.01"' & 'volume <= "0.8"' &'length >= "0.2"' & 'length <= "3"')*distance_to_soma)
print(frame)
pd.DataFrame(frame).to_csv("C:/Users/meowm/Downloads/DiscSpinedata_V1dendriteorigin.csv")

    subject_id  identifier  dendrite_id                  segmentation_name  \
0         L691  Dendrite1a           37  MushroomSpine_1_plane_segmentaton   
1         L691  Dendrite1a           37   StubbySpine_17_plane_segmentaton   
2         L691  Dendrite1a           37   StubbySpine_18_plane_segmentaton   
3         L691  Dendrite1a           37   StubbySpine_19_plane_segmentaton   
4         L691  Dendrite1a           37      ThinSpine_6_plane_segmentaton   
..         ...         ...          ...                                ...   
615       L902   Dendrite2           30      ThinSpine_4_plane_segmentaton   
616       L902   Dendrite2           30      ThinSpine_5_plane_segmentaton   
617       L902   Dendrite2           30      ThinSpine_6_plane_segmentaton   
618       L902   Dendrite2           30      ThinSpine_7_plane_segmentaton   
619       L902   Dendrite2           30      ThinSpine_9_plane_segmentaton   

     genotype sex       species   strain    surgery  \
0    Thy

In [12]:
#Didn't meet criteria because too small

frame = pd.DataFrame(mouse*(session & 'pharmacology = "Vehicle 1% DMSO inject IP 4 weeks after SCI"')*dendrite*(image_segmentation & 'volume <= "0.01"' or 'length < "0.2" ' )*distance_to_soma)
#pd.DataFrame(frame).to_csv("C:/Users/meowm/Downloads/toosmall.csv")
print(len(frame))

97


In [11]:
#Didn't meet criteria because too big
frame = pd.DataFrame(mouse*(session & 'pharmacology = "Vehicle 1% DMSO inject IP 4 weeks after SCI"')*dendrite*(image_segmentation & 'volume > "0.8"' or 'length > "3"')*distance_to_soma)
#pd.DataFrame(frame).to_csv("C:/Users/meowm/Downloads/toobig.csv")
print(len(frame))

24
